<a href="https://colab.research.google.com/github/Vejber/nn-zero-to-hero/blob/master/homeworks/hw3_finetune/hw3_veiber_finetune_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.2 MB/s eta 0:00:00


In [ ]:
import torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import DefaultDataCollator, TrainingArguments, Trainer, \
                         AutoTokenizer, AutoModelForQuestionAnswering, BertTokenizerFast, BertModel, AutoModelForSequenceClassification

Загрузка базовой модели и токенизатора

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Загрузка датасета

In [ ]:
dataset = load_dataset('rajpurkar/squad')

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Функция токенизации

In [ ]:
def tokenize_function(examples):

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results in one example possible giving several features when a context is long, each of those features having a context that overlaps a bit the context of the previous feature.

    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation='only_second',
        # truncate context, not the question
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length',
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop('overflow_to_sample_mapping')

    # The offset mappings will give us a map from token to character position in the original context. This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop('offset_mapping')

    # To label those examples:
    tokenized_examples['start_positions'] = []
    tokenized_examples['end_positions'] = []

    for i, offsets in enumerate(offset_mapping):
        # To label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples['input_ids'][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples['answers'][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers['answer_start']) == 0:
            tokenized_examples['start_positions'].append(cls_index)
            tokenized_examples['end_positions'].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers['answer_start'][0]
            end_char = start_char + len(answers['text'][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples['start_positions'].append(cls_index)
                tokenized_examples['end_positions'].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples['start_positions'].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples['end_positions'].append(token_end_index + 1)

    return tokenized_examples

Применение токенизации к данным

In [ ]:
# Apply the function to our data
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 88524
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10784
    })
})

Подготовка аргументов и модели для обучения

In [ ]:
args = TrainingArguments(
    f'finetune-BERT-QA',
    evaluation_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [ ]:
data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'].select(range(30000)),
    eval_dataset=tokenized_datasets['validation'].select(range(3000)),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Дообучение модели

In [ ]:
# Run the trainer
trainer.train()
# Save the model and tokenizer
trainer.save_model('model_qa')

Epoch,Training Loss,Validation Loss
1,1.040500,1.166683
2,0.692000,1.180865


Функция ответа на вопрос из датасета

In [ ]:
def predicted_answer(instance):
    # Question and context
    context = instance['context']
    question = instance['question']

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get model's output
    with torch.no_grad():
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)

    predicted_answer = tokenizer.convert_tokens_to_string(\
                      tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1])
    )

    print('Вопрос:', question)
    print('Ответ:', predicted_answer)

In [ ]:
predicted_answer(dataset['validation'][0])

Вопрос: Which NFL team represented the AFC at Super Bowl 50?
Ответ: denver broncos


In [ ]:
predicted_answer(dataset['validation'][10])

Вопрос: What day was the Super Bowl played on?
Ответ: february 7, 2016


In [ ]:
predicted_answer(dataset['validation'][20])

Вопрос: What month, day and year did Super Bowl 50 take place? 
Ответ: february 7, 2016


Сравнение ответов модели с ответами из датасета

In [ ]:
instance = dataset['validation'][0]
context = instance['context']
question = instance['question']

given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one
given_answer_start = instance['answers']['answer_start'][0]

# Tokenize the data
inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

inputs = {k: v.to(device) for k, v in inputs.items()}

# Get model's output
with torch.no_grad():
    output = model(**inputs)

# Get the predicted answer
start_idx = torch.argmax(output.start_logits)
end_idx = torch.argmax(output.end_logits)

predicted_answer = tokenizer.convert_tokens_to_string(\
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1])
)
print('Контекст:', context)
print('Вопрос:', question)
print('Ответ из датасета:', predicted_answer)
print('Ответ модели:', given_answer)

Контекст: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Вопрос: Which NFL team represented the AFC at Super Bowl 50?
Ответ из датасета: denver broncos
Ответ модели: Denver Broncos


Функция для оценки

In [ ]:
# Function to evaluate a single instance
def evaluate_instance(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Apply the BERT model
    with torch.no_grad():  # No need to calculate gradients
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower() == given_answer.lower()

Оценка ответов первых 10000 экземпляров в validation

In [ ]:
# Evaluate the a number of instances
correct_count = 0
total_count = 10000

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(dataset['validation'][i], device)

100%|██████████| 10000/10000 [02:43<00:00, 61.11it/s]


Вычисление accuracy

In [ ]:
# Calculate and output the accuracy
accuracy = correct_count / total_count
print(f'Accuracy train: {accuracy * 100:.2f}%')

Accuracy train: 49.50%


Оценка ответов первых 5000 экземпляров в validation датасете

In [ ]:
# Evaluate the a number of instances
correct_count = 0
total_count = 5000

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(dataset['validation'][i], device)

100%|██████████| 5000/5000 [01:34<00:00, 52.69it/s]


In [ ]:
# Calculate and output the accuracy
accuracy = correct_count / total_count
print(f'Accuracy validation: {accuracy * 100:.2f}%')

Accuracy validation: 49.96%


Все вместе

In [ ]:
# Load the dataset
dataset = load_dataset('squad')

# Load the tokenizer and model
model_name = '/content/model_qa'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

def predicted_answer(instance):
    # Question and context
    context = instance['context']
    question = instance['question']

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get model's output
    with torch.no_grad():
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)

    predicted_answer = tokenizer.convert_tokens_to_string(\
                      tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1])
    )

    print('Вопрос:', question)
    print('Ответ:', predicted_answer)

In [ ]:
predicted_answer(dataset['validation'][10])